### MY470 Computer Programming

### Problem Set 4, AT 2023

#### \*\*\* Due 12:00 noon on Monday, November 13 \*\*\*

---
### Writing your own binary classifier evaluation module

Classification is the supervised-learning equivalent of clustering, which we briefly touched on in Problem Set 2. A classifier is used to label an observation as belonging to one of a finite number of categories, or classes. We use labeled training data to build the classifier and then use the classifier to predict the categories that new observations belong to. 

In this problem set, we will build our own module for working with and evaluating binary classifiers and write unit tests for it. We will then use the module to evaluate k-nearest neighbor (KNN) classification on social data. We will use the scikit-learn package to run the KNN algorithm.  

We will use data from the file `../data/house-votes-84.data`. The file contains information on the voting record from the 1984 United States Congress. Our goal will be to predict whether the voter is a Democrat based on how they voted on 16 separate occasions. The data are obtained from the [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/index.php) and you can find more information about them in the file `../data/house-votes-84.names` as well as [here](http://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records).

#### Hints

**This problem set aims to test your mastery of function specifications, exceptions, assertions, and unit testing. Make sure your code handles different possible inputs that are not explicitly excluded in the function specification and deals appropriately with likely errors.**   

For this problem set, **you are allowed to use GitHub Copilot**, which will help you with writing out all the code you are expected to. Please follow the instructions from lecture and class to install it in VS Code.

### Problem 1: Function to split data into training and testing sets

Create a .py file called `class_eval.py` and in it write a function called `split_training_testing()` that takes as arguments a list of data points and a number indicating the percent of data to be used for testing. The function should randomly assign data points to either the training or testing sets, where the size of the sets is determined by the percent passed as an argument. The function should return the training and testing sets, in that particular order. The term "set" here comes from machine learning and you should not take it literally. The two outputs your function returns should be in fact of type `list`. 

#### Hints

You can use this notebook to write and test your code but for your final submission, please make sure the code is in `class_eval.py` and not here.


### Problem 2: Function to estimate values from confusion matrix

In the file `class_eval.py`, write a function called `confusion_matrix()` that takes as arguments a list of predicted values, a list of actual values, and a value indicating which class is the positive class and returns the number of true positives (TP), false positives (FP), true negatives (TN), and false negatives (FN), in that specific order. Here is how a confusion matrix looks like, assuming that 1 is the positive class and 0 is the negative class:

|             | Actual 1 | Actual 0   
|:-----------:|:--------:|:----------------------
| **Predicted 1** | True positive | False positive             
| **Predicted 0** | False negative | True negative


### Problem 3: Functions to estimate evaluation metrics

In the file `class_eval.py`, write five functions that estimate the accuracy, sensitivity, specificity, positive predictive value, and negative predictive value. 

Accuracy is the proportion of predictions that are correct, namely: $accuracy = \frac{TP + TN}{TP + TN + FP + FN}$.

Sensitivity (also known as recall) is the proportion of positives that are correctly identified as such: $sensitivity = \frac{TP}{TP + FN}$.

Specificity (also known as precision) is the proportion of negatives that are correctly identified as such: $specificity = \frac{TN}{TN + FP}$.

Positive predictive value is the probability that a data point identified as positive is truly such: $positive \ predictive\ value = \frac{TP}{TP + FP}$.

Negative predictive value is the probability that a data point identified as negative is truly such: $negative\ predictive\ value = \frac{TN}{TN + FN}$.

The functions should take the required arguments and return the estimate. Use the following names for your functions: `accuracy()`, `sensitivity()`, `specificity()`, `pos_pred_val()`, `neg_pred_val()`.

#### Hints

The functions should return `float('nan')` if they encounter division by 0.


### Problem 4: Function to print evaluation metrics

In the file `class_eval.py`, write a function called `print_eval_metrics()` that takes as arguments a list of predicted values, a list of actual values, and a value indicating which class is the positive class and prints the accuracy, the sensitivity, the specificity, the positive predictive value, and the negative predictive value.


### Problem 5: Unit tests for the classification evaluation module

Create a .py file called `tests.py` and use the module `unittest` to write tests for the appropriate functions above. Follow this week's materials from lecture and class to identify informative testing values. You can read more about the `unittest` module [here](https://docs.python.org/3/library/unittest.html).  

#### Hints

Feel free to look at the testing modules of recognized Python packages for inspiration. 


---
## Evaluating your module using KNN on real data

The code below uses the functions you have written and scikit-learn to run the KNN algorithm on a dataset and evaluate its performance. **Make sure you submit code that runs with the code below. The output we have provided also shows you how `print_eval_metrics()` from Problem 4 should behave.**

We first define a function called `get_data()` that opens the file "house-votes-84.data" and returns all the data in a list, where each item is a list starting with the political affiliation of the congress member and then including the voting decisions of that congress member. The voting decisions are saved as a list of `0`'s, `0.5`'s, and `1`'s, where `0` stands for `'y'` in the data, `1` stands for `'n'`, and `0.5` stands for `'?'`. 

We get the data and split them into a 80% training and 20% testing set. We then fit the KNN classifier on the training set and predict the labels for the testing set. We use the predicted values and the actual values from the testing set to evaluate the algorithm's performance, where we assume that the positive value is "democrat". If curious, you can read more about how to use scikit-learn's KNN [here](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html). 

In order to run the code, you may first need to install scikit-learn. There are two ways to do this:

1. Open a Terminal window and type: `conda install scikit-learn`
2. Alternatively, open Anaconda Navigator, go to Environments, select "All" from the drop-down menu and type "scikit-learn" in the field "Search Packages". Then select the package and click on "Apply". 


In [1]:
import random
import class_eval
from sklearn.neighbors import KNeighborsClassifier

FPATH = '../data/house-votes-84.data'

# Fix the random seed for predictability, if needed
random.seed(0)

def get_data(fpath):
    """Opens file house-votes-84.data and returns a list with 
    labels (political affiliations) and a list with 
    feature vectors (voting decisions). Voting decisions 
    are represented with 1 for yes, 0 for no, and 0.5 for neither.
    """
    relabel = {'y': 1, 'n': 0, '?': 0.5}
    data = []
    for line in open(fpath, 'r'):
        strlst = line.strip().split(',')
        toappend = [strlst[0]] + [relabel[i] for i in strlst[1:]]
        data.append(toappend)
    return data 

# Get the data
data = get_data(FPATH)

# Split it into training and testing sets and separate labels from feature vectors
train, test = class_eval.split_training_testing(data, p_test=20)
train_labels = [i[0] for i in train]
train_features = [i[1:] for i in train]
test_actual_labels = [i[0] for i in test]
test_features = [i[1:] for i in test]

# Make an instance of the KNN classifier and fit a model to the training data
neigh = KNeighborsClassifier(n_neighbors=11)
neigh.fit(train_features, train_labels) 

# Predict the labels for the test data and evaluate the performance
test_pred_labels = neigh.predict(test_features)

# The predict() method returns an object of type numpy.ndarray, so
# we will transform it to list to fit the function specification
class_eval.print_eval_metrics(list(test_pred_labels), test_actual_labels, 'democrat')

# This routine is meant for testing purposes only.
# In an actual analysis, we will look more systematically for a k 
# that maximizes the model's accuracy. We will then use multiple rounds 
# of random partitioning and average the model's performance over all rounds.

Accuracy: 0.9540229885057471
Sensitivity: 0.9285714285714286
Specificity: 1.0
Positive predictive value: 1.0
Negative predictive value: 0.8857142857142857


---

### Evaluation

| Problem | Mark     | Comment   
|:-------:|:--------:|:----------------------
| 1       |   1/2    |   `random.shuffle` in place is dangerous here since you change the original object.          
| 2       |   2/2    | Good
| 3       |   1/2    | Better to try/except with ZeroDivisionError than if/else to return float(‘nan’).
| 4       |   1/1    | Good
| 5       |   4/5    | Should use self.assertAlmostEqual when testing float output
| Legibility |   1/2    | 2 - stick to lowercase variable names. 
| Modularity |   4/4    | Good
| Efficiency |   2/2    | Good
|**Total**|**16/20**  | Very good